In [1]:
# FILTER DATA
from parse_truth_ALL import main as parse_truth_all
import collections
from algorithm_output_parser import process_algorithm_output
from algorithm_output_parser import toboggan_output_parser
from algorithm_output_parser import toboggan_clean_output_parser
from algorithm_output_parser import catfish_output_parser

# [1] COLLECT ALL GROUND TRUTH DATA
from algorithm_output_parser import toboggan_output_parser


froot_indices = [0,1,2,3]
froots = ['zebra', 'mouse', 'human', 'salmon']
datadirs = ['zebrafish', 'mouse', 'human', 'salmon']
    
print("\\toprule")
print("dataset &  instances &  non-trivial &  avg nodes  &  avg deg & avg $k$  &  optimal &  non-optimal \\\\")
print("\\midrule")


total_instances = 0
total_nontrivials = 0
total_optimals = 0
total_nonoptimals = 0
total_contracted_nodes = 0
total_contracted_ave_deg = 0
total_orig_nodes = 0
total_orig_ave_deg = 0
total_ave_kgtrue = 0

total_timeouts = 0

total_cutset_eq_opt = 0
total_improv_eq_opt = 0


for froot_idx in froot_indices:
    froot = froots[froot_idx]
    #print("working on {}...".format(froot))
    
    num_instances = 0
    num_nontrivial_instances = 0

    num_gt_optimal_size = 0
    num_gt_wrong_size = 0
    num_timeouts = 0

    ave_k_gtrue = 0
    ave_num_nodes_contracted = 0
    ave_deg_contracted = 0
    ave_num_nodes_orig = 0
    ave_deg_orig = 0

    this_cutset_eq_opt = 0
    this_improv_eq_opt = 0

    # COUNT RELEVANT INFO

    datadirending = datadirs[froot_idx]
    datadir = '/home/kyle/data/rnaseq/' + datadirending + '/'

    # [0] Get data from toboggan
    inputfile = "./data/all-" + froot + ".txt"
    
    truth_dict = {}
    with open('./data/truth-size-' + froot + '.txt', 'r') as infile:
        for line in infile:
            parts = line.strip().split()
            key = parts[0] + ' ' + parts[1]
            truth_dict[key] = int(parts[2])
    
    temp_dict = collections.defaultdict(int)
    
    with open(inputfile,'r') as infile:
        for line in infile:
            
            temp_parts = line.strip().split()
            temp_key = temp_parts[0].split('.')[0] + ' ' + temp_parts[1]

            if temp_dict[temp_key] == 0:
                num_instances += 1
                n_orig = int(temp_parts[2])
                m_orig = int(temp_parts[3])
                n_red = int(temp_parts[4])
                m_red = int(temp_parts[5])
                # k_gtrue = int(temp_parts[6])
                k_gtrue = truth_dict[temp_key]

                if k_gtrue > 1:
                    num_nontrivial_instances += 1
                    ave_k_gtrue += k_gtrue
                    
                else:  # skip trivial instances
                    continue


                k_cutset = int(temp_parts[7])
                k_improv = int(temp_parts[8])
                
                if k_cutset == k_gtrue:
                    this_cutset_eq_opt += 1
                if k_improv == k_gtrue:
                    this_improv_eq_opt += 1

                k_opt = temp_parts[9]
                if k_opt != 'None':
                    k_opt = int(k_opt)
                    if k_opt == k_gtrue:
                        num_gt_optimal_size += 1
                    elif k_opt < k_gtrue:
                        num_gt_wrong_size += 1
                else:
                    num_timeouts +=1

                ave_num_nodes_contracted += n_red
                ave_deg_contracted += 2*(m_red/n_red)
                ave_num_nodes_orig += n_orig
                ave_deg_orig += 2*(m_orig/n_orig)
                
                temp_dict[temp_key] = k_opt
            else:
                k_gtrue = truth_dict[temp_key]
                k_opt = temp_parts[9]
                if k_opt != temp_dict[temp_key]:
                    if k_opt != 'None':
                        k_opt = int(k_opt)
                        if k_opt == k_gtrue:
                            num_gt_optimal_size += 1
                        elif k_opt < k_gtrue:
                            num_gt_wrong_size += 1
                
            
            """
            row = [ filename, instancenum, n, m, n_red, m_red,
                           k_groundtruth, cutset_bound, improved_bound, k_opt,
                           t_w, t_path, timeout_flag, timeout_limit, graphname]
            """
    
    total_ave_kgtrue += ave_k_gtrue
    total_contracted_nodes += ave_num_nodes_contracted
    total_contracted_ave_deg += ave_deg_contracted
    total_orig_nodes += ave_num_nodes_orig
    total_orig_ave_deg += ave_deg_orig
    
    total_cutset_eq_opt += this_cutset_eq_opt
    total_improv_eq_opt += this_improv_eq_opt

    ave_k_gtrue = ave_k_gtrue / num_nontrivial_instances
    ave_num_nodes_contracted = ave_num_nodes_contracted / num_nontrivial_instances
    ave_deg_contracted = ave_deg_contracted / num_nontrivial_instances
    ave_num_nodes_orig = ave_num_nodes_orig / num_nontrivial_instances
    ave_deg_orig = ave_deg_orig / num_nontrivial_instances
    
    total_instances += num_instances
    total_nontrivials += num_nontrivial_instances
    total_optimals += num_gt_optimal_size
    total_nonoptimals += num_gt_wrong_size
    total_timeouts += num_timeouts
        
    print("\\{} & {:d} & {:d} & {:3.2f} & {:3.2f} & {:3.2f} &  {:3.3f}\\% & {:3.3f}\\% \\\\ % {}".format(froot,
        num_instances, num_nontrivial_instances, ave_num_nodes_orig, ave_deg_orig, ave_k_gtrue,
        100*num_gt_optimal_size/(num_nontrivial_instances), 100*num_gt_wrong_size/num_nontrivial_instances,
        num_timeouts, len(truth_dict)))

print("\\midrule")
print("{} & {:d} & {:d} & {:3.2f} & {:3.2f} & {:3.2f} & {:3.3f}\\% & {:3.3f}\\% \\\\ % {}".format('all',
        total_instances, total_nontrivials, total_orig_nodes/total_nontrivials,
        total_orig_ave_deg/total_nontrivials , total_ave_kgtrue/total_nontrivials,
        100*total_optimals/(total_nontrivials), 100*total_nonoptimals/total_nontrivials, total_timeouts))

print("")
print("k_cut: {},  k_imp: {}".format(total_cutset_eq_opt/total_nontrivials, total_improv_eq_opt/total_nontrivials) )

\toprule
dataset &  instances &  non-trivial &  avg nodes  &  avg deg & avg $k$  &  optimal &  non-optimal \\
\midrule
\zebra & 1549373 & 445880 & 18.49 & 2.27 & 2.32 &  99.907\% & 0.053\% \\ % 209
\mouse & 1316058 & 473185 & 18.67 & 2.37 & 2.75 &  99.401\% & 0.074\% \\ % 3075
\human & 1169083 & 529523 & 18.79 & 2.41 & 2.83 &  99.490\% & 0.043\% \\ % 3164
\salmon & 13300893 & 7153472 & 20.54 & 2.55 & 3.74 &  94.391\% & 0.035\% \\ % 398752
\midrule
all & 17335407 & 8602060 & 20.22 & 2.52 & 3.55 & 95.266\% & 0.039\% \\ % 405200

k_cut: 0.9653462077688367,  k_imp: 0.9842896933990231


In [20]:
(17335407 - 8602060) / 17335407

0.5037866719829537